# Dictionaries

In [716]:
import plotly.express as px
import random

# Classes and functions

#### Node

In [717]:
class Node():
    
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self) -> str:
        return f"{self.position}"


#### Agent

In [718]:
class Agent:
    def __init__(self, start_position, end_position):
        self.start_node = Node(None, start_position)
        self.end_node = Node(None, end_position)
        self.current_node = Node(None, start_position)

        self.open_list = []
        self.close_list = []
        self.path = []

#### Random points Generator

In [719]:
# Define a function to generate random start and end points
def generate_random_points(maze_size):
    while True:
        start_x = random.randint(0, maze_size - 1)
        start_y = random.randint(0, maze_size - 1)
        end_x = random.randint(0, maze_size - 1)
        end_y = random.randint(0, maze_size - 1)
        if (start_x, start_y) != (end_x, end_y):
            return (start_x, start_y), (end_x, end_y)


#### Agent Generator

In [720]:
def generate_agents(number_of_agents, maze_size):
    
    agents = []
    for _ in range(number_of_agents):
        # Update start and end points for each agent
        start, end = generate_random_points(maze_size)
        agent = Agent(start, end)
        agents.append(agent)
    return agents


#### Check if all agents arrived

In [721]:
def reach_final(agents):
    for agent in agents:
        if agent.current_node != agent.end_node:
            return(False)
    return(True)

# Main Code

In [722]:
def astar(maze, agents):
    
    # Check if Start and End positions are the same
    counter = 0
    for agent in agents:
        counter += 1
        if(agent.start_node.position == agent.end_node.position):
            raise Exception("Start and End positions of Agent " + str(counter) +" are same")
        


    new_maze = maze
    maze_x = len(maze)-1
    maze_y = len(maze[0])-1
    
    counter = 0
    for agent in agents:
        counter += 1
        if(maze[agent.start_node.position[0]][agent.start_node.position[1]] == 0): 
            raise Exception("Start position of Agent "+str(counter)+ " is on Block in the position: " )
        if(maze[agent.end_node.position[0]][agent.end_node.position[1]] == 0): 
            raise Exception("End position of Agent "+str(counter)+" is on Block in the position: ")
    

    # Add the agent startnode
    counter = 0
    for agent in agents:
        counter += 1
        agent.path.append(agent.current_node)
        # print("Agent "+ str(counter)  + " is at: ", agent.current_node.position, "With the cost of :" , agent.current_node.g , "Till now and the heuristic of: " , agent.current_node.h)
        agent.close_list.append(agent.start_node)

    # Loop until you find the end
    while True:
        # Found the goal
        if reach_final(agents):
            final_paths = []
            
            for agent in agents:
                final_path = []  
                goal = agent.path[-1]
                while goal is not None:
                    final_path.append(goal)
                    goal = goal.parent
                final_paths.append(final_path)

            for i in range(len(final_paths)):
                final_paths[i] = final_paths[i][::-1]
            
            try:
                longest_list_length = max(map(len, final_paths))
                print("Longest list length", longest_list_length)
                index_position_agents_conflicts_map = {}
                for index in range(longest_list_length):
                    index_position_agents_conflicts_map[index] = {}
                    conflict_node_map = {}
                    for list_index, list_context in enumerate(final_paths, 0):
                        if index < len(list_context) - 1:
                            element = list_context[index]
                            if element.parent.position not in conflict_node_map:
                                conflict_node_map[element.parent.position] = []
                            conflict_node_map[element.parent.position].append(list_index)
                    for conflict_position,conflict_position_agents in conflict_node_map.items():
                        if len(conflict_position_agents) > 1:
                            index_position_agents_conflicts_map[index][conflict_position] = conflict_position_agents
                    # print(index, index_position_agents_conflicts_map[index])
            except:
                print("An exception occurred on index ", index)
                
            for index_postions_agents_conflicts in index_position_agents_conflicts_map.items():
                conflict_node_parent_index = index_postions_agents_conflicts[0]
                if len(index_postions_agents_conflicts[1]) > 0:  
                    list_of_heuroestics = []
                    counter = -2
                    for conflict_position,conflict_position_agents in index_postions_agents_conflicts[1].items():
                        for conflicted_agent in conflict_position_agents:
                            counter += 1
                            list_of_heuroestics.append(final_paths[conflicted_agent][conflict_node_parent_index].h)
                        x = list_of_heuroestics.index(min(list_of_heuroestics))
                        stay_node = Node(final_paths[x + counter][conflict_node_parent_index], final_paths[x + counter][conflict_node_parent_index].position)
                        final_paths[x + counter][conflict_node_parent_index] = stay_node
                        final_paths[x + counter].insert(conflict_node_parent_index, stay_node)
                        
                
            final_path_primes = []    
                    
            for final_path in final_paths:
                final_path_prime = []
                goal = final_path[-1]
                while goal is not None:
                    final_path_prime.append(goal)
                    goal = goal.parent
                final_path_primes.append(final_path)

            counter = 2
            for final_path_prime in final_path_primes:
                final_path_prime = final_path_prime[::-1]
                print("agent ", counter - 1," has the path",final_path_prime) # Print reversed path
                for path_position in final_path_prime:
                    new_maze[path_position.position[0]][path_position.position[1]] = counter * 2
                counter +=1
            
            return(new_maze)
        
        counter = 1
        for agent in agents:
            counter += 1
            if agent.current_node != agent.end_node:
            # Generate children
                for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0),(-1, -1), (1, 1), (-1, 1), (1, -1)]: # Adjacent squares

                    # Get node position
                    adjacent_position = (agent.current_node.position[0] + new_position[0], agent.current_node.position[1] + new_position[1])

                    # Make sure within range
                    if adjacent_position[0] > maze_x or adjacent_position[0] < 0 or adjacent_position[1] > maze_y or adjacent_position[1] < 0:
                        continue

                    # Make sure walkable terrain
                    if maze[adjacent_position[0]][adjacent_position[1]] == 0:
                        continue

                    # Create new node
                    new_node = Node(agent.current_node, adjacent_position)

            
                    # Child is on the closed list
                    if new_node in agent.close_list:
                        continue

                    # Create the f, g, and h values
                    new_node.g = agent.current_node.g + 1
                    new_node.h = ((new_node.position[0] - agent.end_node.position[0]) ** 2) + ((new_node.position[1] - agent.end_node.position[1]) ** 2)
                    new_node.f = new_node.g + new_node.h

    
                    # new_node is already in the open list
                    if new_node not in agent.close_list:
                        if new_node in agent.open_list:
                            conflicted_agent = agent.open_list.index(new_node)
                            duplicated = agent.open_list[conflicted_agent]
                            if new_node.g < duplicated.g:
                                agent.open_list.append(new_node)
                        else:
                            agent.open_list.append(new_node)
            
            if len(agent.open_list) > 0 and agent.current_node != agent.end_node:
                # Get the agent 1 current node
                agent.current_node = agent.open_list[0]
                for item in agent.open_list:
                    # print("openList:" ,item.position[0],  item.position[1] )
                    # new_maze[item.position[0]][item.position[1]] = (counter * 2) + 1 # change openlist items colors
                    if item.f < agent.current_node.f:
                        agent.current_node = item
                # print("Agent ", counter - 1 ," is at: ", agent.current_node.position, "With the cost of :" , agent.current_node.g , "Till now and the heuristic of: " , agent.current_node.h)
                # Pop current off open list, add to closed list
                agent.open_list.remove(agent.current_node)
                agent.path.append(agent.current_node)
                agent.close_list.append(agent.current_node)

In [723]:
def create_maze(maze_size, zeros_number):
    # Define the size of the maze
    maze_size = 15

    # Create a new maze with all fives
    new_maze = [[5] * maze_size for _ in range(maze_size)]

    # Generate 20 random coordinates and set them to zero
    zeros_number = 20
    zeros_generated = 0

    while zeros_generated < zeros_number:
        x = random.randint(0, maze_size - 1)
        y = random.randint(0, maze_size - 1)
        if new_maze[x][y] == 5:
            new_maze[x][y] = 0
            zeros_generated += 1
    maze = []
    # Print the new maze
    for row in new_maze:
        maze.append(row)
    return maze


In [724]:
def main():

    maze = [[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]
    maze_size = 15
    zeros_number = 40
    number_of_agents = 20
    # maze = create_maze(maze_size, zeros_number)
    agents = generate_agents(number_of_agents, maze_size)
    for agent in agents:
        print("start and end positions of agent ",agents.index(agent) + 1,"is: ",agent.start_node.position, agent.end_node.position)

    agents[0] = Agent((10, 8) ,(9, 3))
    agents[0] = Agent((10, 8) ,(9, 3))
    new_maze = astar(maze, agents)
    
    

    fig = px.imshow(new_maze, color_continuous_midpoint=0.0 , range_color= [-10,42], color_continuous_scale = 'Picnic' )
    fig.show()


In [725]:
main()

start and end positions of agent  1 is:  (1, 2) (12, 7)
start and end positions of agent  2 is:  (0, 13) (5, 4)
start and end positions of agent  3 is:  (8, 8) (0, 14)
start and end positions of agent  4 is:  (10, 7) (0, 10)
start and end positions of agent  5 is:  (13, 4) (0, 5)
start and end positions of agent  6 is:  (11, 13) (4, 14)
start and end positions of agent  7 is:  (10, 10) (5, 3)
start and end positions of agent  8 is:  (13, 1) (2, 3)
start and end positions of agent  9 is:  (1, 11) (9, 11)
start and end positions of agent  10 is:  (14, 7) (12, 11)
start and end positions of agent  11 is:  (13, 8) (7, 5)
start and end positions of agent  12 is:  (13, 5) (8, 14)
start and end positions of agent  13 is:  (4, 2) (8, 9)
start and end positions of agent  14 is:  (12, 0) (6, 13)
start and end positions of agent  15 is:  (3, 10) (2, 0)
start and end positions of agent  16 is:  (8, 7) (6, 10)
start and end positions of agent  17 is:  (10, 10) (6, 14)
start and end positions of age